In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import numpy as np
import os
#! pip install wget
import wget
!pip install pyproj
import pyproj
import folium
import json
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
#!conda install shapely
from shapely.geometry import shape, Point

In [ ]:
# get data
try:
    df = pd.read_csv('EWR_Ortsteile_2016-06-30.csv', encoding='latin-1', sep = ';')
except:
    wget.download("https://www.statistik-berlin-brandenburg.de/opendata/EWR_Ortsteile_2016-06-30.csv")
#df_dem

df.dtypes

In [ ]:
try:
    df = pd.read_csv('EWR_Ortsteile_2016-06-30.csv', encoding='latin-1', sep = ';')
except:
    wget.download("https://www.statistik-berlin-brandenburg.de/opendata/EWR_Ortsteile_2016-06-30.csv")

In [57]:

berlin_json = r'bezirksgrenzen.geojson' # geojson file

# create a plain world map
map_berlin = folium.Map(location=[52.5200, 13.4050], zoom_start=10)


folium.GeoJson(
    berlin_json,
    name='geojson'
).add_to(map_berlin)

map_berlin

In [3]:
df.head()

NameError: name 'df' is not defined

In [ ]:
df_dem = df.groupby(['Bez-Name', 'Altersgr']).sum()
df_dem.head(10)


In [ ]:
df_dem = df[['Bez-Name','Häufigkeit']].groupby(['Bez-Name']).sum().reset_index()
df_dem=df_dem.rename(columns = {'Bez-Name': 'District', 'Häufigkeit' : 'Population'} )
df_dem.head()

In [ ]:
map_berlin_pop = folium.Map(location=[52.5200, 13.4050], zoom_start=11)
map_berlin_pop.choropleth(
    geo_data=berlin,
    data=df_dem,
    columns=['District', 'Population'],
    key_on='feature.properties.Gemeinde_name',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Berlin Population',
    threshold_scale=[200000,250000,300000,350000,400000]
)

map_berlin_pop

In [ ]:
[200000,225000,250000,275000,300000,325000,350000,400000]

In [ ]:
df_dem = df[['Bez-Name','Häufigkeit','Altersgr']].groupby(['Bez-Name', 'Altersgr']).sum().reset_index()
df_dem=df_dem.rename(columns = {'Bez-Name': 'District', 'Altersgr': 'AgeGroup', 'Häufigkeit' : 'Population'} )

df_dem.head(10)

In [ ]:
df=df.rename(columns = {'Bez-Name': 'District', 'Altersgr': 'AgeGroup', 'Häufigkeit' : 'Population'} )
df_dem = pd.pivot_table(df[['District','Population','AgeGroup']], values='Population', index=['District'],columns=['AgeGroup'],aggfunc=np.sum)
df_dem.columns[0:3]
df_dem['0-15'] = df_dem[df_dem.columns[0:3]].sum(axis = 1)
df_dem['15-30'] = df_dem[df_dem.columns[3:6]].sum(axis = 1)
df_dem['30-50'] = df_dem[df_dem.columns[6:10]].sum(axis = 1)
df_dem['50-65'] = df_dem[df_dem.columns[10:13]].sum(axis = 1)
df_dem['>65'] = df_dem[df_dem.columns[13:20]].sum(axis = 1)
df_dem['Total'] = df_dem[df_dem.columns[1:20]].sum(axis = 1)

#df_dem['Senior'] = df_dem[['15_20','20_25' ,'25_30']].sum(axis = 1)
df_dem=df_dem[['0-15','15-30','30-50','50-65','>65','Total']]
df_dem

In [ ]:
df_dem_rel = df_dem[['0-15','15-30','30-50','50-65','>65']].div(df_dem['Total'], axis=0).round(2)
df_dem_rel

In [6]:
address = 'Berlin Germany'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
berlin = [location.latitude, location.longitude]


In [7]:
def lonlat2xy(lat,lon):
    proj_coord = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_coord, proj_xy, lon,lat)
    return xy[0], xy[1]
    
def xy2lonlat(x,y):
    proj_coord = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    coord = pyproj.transform(proj_xy, proj_coord, x, y)
    return coord[0], coord[1]

# convert Berlon Coordinates
x, y = lonlat2xy(berlin[0], berlin[1])
lon, lat = xy2lonlat(x,y)
print('Berlin center coordinate, Lon= {}; Lat= {}'.format(berlin[1], berlin[0]))
print('Berlin center transform to UTM, X= {}; Y= {}'.format(x,y))
print('Berlin center reverse transform, Lon= {}; Lat= {}'.format(lon, lat))


Berlin center coordinate, Lon= 13.3888599; Lat= 52.5170365
Berlin center transform to UTM, X= 390676.87103088724; Y= 5819766.874314443
Berlin center reverse transform, Lon= 13.3888599; Lat= 52.5170365


In [88]:
x_center, y_center = lonlat2xy(berlin[0], berlin[1])
XX = np.arange(-20,21,2)*1000 + x_center
YY = np.arange(20,-21,-2)*1000 + y_center

In [89]:
Lats =[]
Lons = []

for y in YY:
    for x in XX:
        lon, lat =xy2lonlat(x,y)
        Lats.append(lat)
        Lons.append(lon)
                
    

In [115]:
df_points = pd.DataFrame(columns = ['Latitude','Longitude', 'District'])

map_berlin = folium.Map(location=[52.5200, 13.4050], zoom_start=10)
folium.GeoJson(
    berlin_json,
    name='geojson'
).add_to(map_berlin)



for lon, lat in zip(Lons ,Lats):
    point = Point(lon, lat)
    for feature in js['features']:
        polygon = shape(feature['geometry'])
        #print(feature['properties']['Gemeinde_name'])
        if polygon.contains(point):
            #print(feature['properties']['Gemeinde_name'])
            df_points = df_points.append({'Latitude': lat, 'Longitude': lon, 'District':feature['properties']['Gemeinde_name'] 
                                         }, ignore_index=True)

            folium.features.CircleMarker(
                [lat, lon],
                radius=2, # define how big you want the circle markers to be
                color='yellow',
                fill=True,
                fill_color='yellow',
                popup = 'Lat={:5.2f}; Lon={:5.2f}'.format(lat, lon),
                fill_opacity=0.9
            ).add_to(map_berlin)
            

map_berlin

In [116]:
df_points=df_points.sort_values(by = 'District').reset_index()
df_points=df_points[['Latitude','Longitude', 'District']]
df_points.head()

,Latitude,Longitude,District
0,52.533772,13.299783,Charlottenburg-Wilmersdorf
1,52.480274,13.331299,Charlottenburg-Wilmersdorf
2,52.478995,13.242992,Charlottenburg-Wilmersdorf
3,52.478554,13.213558,Charlottenburg-Wilmersdorf
4,52.498247,13.330618,Charlottenburg-Wilmersdorf
